# Modern database capabilities in Azure SQL Database

This is a SQL Notebook, which allows you to separate text and code blocks and save code results. Azure Data Studio supports several languages, referred to as kernels, including SQL, PowerShell, Python, and more.

In this activity, you'll explore how Azure SQL Database is great for modern scenarios that require JSON and/or geospatial support by using T-SQL to analyze both.

## Set up: Connect to `bus-db`

At the top of the window, select **Select Connection** \> **Change Connection** next to "Attach to".

Under _Recent Connections_ select your `bus-db` connection.

You should now see it listed next to _Attach to_.

## Part 1: Explore JSON support

If you want to start over at any point, run the below cell to delete the temporary tables. Otherwise, you can skip it.

In [1]:
DROP TABLE IF EXISTS #t;
DROP TABLE IF EXISTS #g;
DROP TABLE IF EXISTS #r;

Commands completed successfully.

Total execution time: 00:00:00.035

To take a look at an example, let's create a payload which contains two bus data points. This JSON format is similar to what will be ultimately pulled from the real-time data source. Our goal in this section is to add the received bus geolocation data and check if the buses are inside any predefined GeoFence.

Once you declare a payload, you can use it to insert the data into a temporary table `#t`. Notice how `openjson(@payload)` allows you to parse the JSON very easily with T-SQL.

One other thing to call out from the select statement below is the `GEOGRAPHY::Point([Latitude], [Longitude], 4326)` which is able to take in latitudes and longitudes and convert it to a spatial reference identifier (SRID) which applies to a certain standard (in this case `4326` is used). More on geospatial later in this activity.

In [2]:
DECLARE @payload NVARCHAR(max) = N'[{
		"DirectionId": 1,
		"RouteId": 100001,
		"VehicleId": 1,
		"Position": {
			"Latitude": 47.61705102765316,
			"Longitude": -122.14291865504012 
		},
		"TimestampUTC": "20201031"
	},{
        "DirectionId": 1,
		"RouteId": 100531,
		"VehicleId": 2,
		"Position": {
			"Latitude": 47.61346156765316,
			"Longitude": -122.14291784492805
		},
		"TimestampUTC": "20201031"
}]';

SELECT
	[DirectionId], 
	[RouteId], 
	[VehicleId], 
	GEOGRAPHY::Point([Latitude], [Longitude], 4326) AS [Location], 
	[TimestampUTC]
INTO #t
FROM
	openjson(@payload) WITH (
		[DirectionId] INT,
		[RouteId] INT,
		[VehicleId] INT,
		[Latitude] DECIMAL(10,6) '$.Position.Latitude',
		[Longitude] DECIMAL(10,6) '$.Position.Longitude',
		[TimestampUTC] DATETIME2(7)
	);

(2 rows affected)

Total execution time: 00:00:00.096

Now that you've inserted data into `#t`, take a look at the results. Azure SQL Database is able to take in the JSON data and turn it into a table without difficulty.

In [3]:
select * from #t;

(2 rows affected)

Total execution time: 00:00:00.052

DirectionId,RouteId,VehicleId,Location,TimestampUTC
1,100001,1,0xE6100000010C677BF486FBCE474088BEBB9525895EC0,2020-10-31 00:00:00.0000000
1,100531,2,0xE6100000010C5CAB3DEC85CE47409F008A9125895EC0,2020-10-31 00:00:00.0000000


## Part 2: Explore geospatial support

You saw briefly how the longitude and latitude were converted to a point using \`GEOGRAPHY::Point()\`. In the previous statement you see it as a long string of letters and numbers. By using `ToString()`, you can easily see the point values.

In [4]:
SELECT [VehicleId], [Location].ToString() AS Location FROM #t;

(2 rows affected)

Total execution time: 00:00:00.124

VehicleId,Location
1,POINT (-122.142919 47.617051)
2,POINT (-122.142918 47.613462)


You can navigate to [https://clydedacruz.github.io/openstreetmap-wkt-playground](https://clydedacruz.github.io/openstreetmap-wkt-playground), clear the sample, enter in one of the points and select **Plot Shape** to see the point displayed on a map.

You can do other things with the geospatial support, a common one might be to find the distance between, in this case, the two buses.

In [5]:
declare @bus1 geography;
declare @bus2 geography;
select @bus1 = [Location] from #t where VehicleId = 1;
select @bus2 = [Location] from #t where VehicleId = 2;
select @bus1.STDistance(@bus2) as DistanceInMeters;

(1 row affected)

Total execution time: 00:00:00.082

DistanceInMeters
399.03519789173953


In addition to points, you can also define and store polygons on Earth's surface. This is what we have been referring to as a GeoFence. At the same URL as before, you can replace the `POINT` values with, for example, `POLYGON ((-122.14359028995352 47.618245191245848, -122.14360975757847 47.616519550427654, -122.13966755206604 47.616526111887509, -122.13968701903617 47.617280676597375, -122.142821316476 47.617300360798339, -122.142821316476 47.618186139853435, -122.14359028995352 47.618245191245848))` and see the shape on the map.

This shape represents the GeoFence where you might want to be notified that your bus is entering or exiting. Azure SQL Database also supports using the `POLYGON` format to add that data to a table, as shown below.

In [6]:
SELECT * INTO #g 
FROM (VALUES(
        CAST('Overlake Stop' AS NVARCHAR(100)),
        GEOGRAPHY::STGeomFromText('POLYGON ((-122.14359028995352 47.618245191245848, -122.14360975757847 47.616519550427654, -122.13966755206604 47.616526111887509, -122.13968701903617 47.617280676597375, -122.142821316476 47.617300360798339, -122.142821316476 47.618186139853435, -122.14359028995352 47.618245191245848))',4326)
    ))
    AS s ([BusStop], [GeoFence])
SELECT * FROM #g

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.121

BusStop,GeoFence
Overlake Stop,0xE6100000010407000000B4A78EA822CF4740E8D7539530895EC03837D51CEACE4740E80BFBE630895EC0ECD7DF53EACE4740E81B2C50F0885EC020389F0D03CF4740E99BD2A1F0885EC00CB8BEB203CF4740E9DB04FC23895EC068C132B920CF4740E9DB04FC23895EC0B4A78EA822CF4740E8D7539530895EC001000000020000000001000000FFFFFFFF0000000003


Now that you have defined a few points and a GeoFence, you might want to know if and when a bus is located from within the GeoFence. With Azure SQL Database, that is easy to do.

In [7]:
SELECT
    t.DirectionId,
    t.RouteId,
    t.VehicleId,
    GEOGRAPHY::STGeomCollFromText('GEOMETRYCOLLECTION(' + t.[Location].ToString() + ', ' + g.[GeoFence].ToString() +')',4326).ToString() as [WKT],
    t.[Location].STWithin(g.[GeoFence]) as InGeoFence
INTO #r 
FROM #t AS t 
CROSS JOIN #g AS g 
WHERE g.[BusStop] = 'Overlake Stop';

SELECT * FROM #r;

(2 rows affected)

(2 rows affected)

Total execution time: 00:00:00.118

DirectionId,RouteId,VehicleId,WKT,InGeoFence
1,100001,1,"GEOMETRYCOLLECTION (POINT (-122.142919 47.617051), POLYGON ((-122.14359028995352 47.618245191245848, -122.14360975757847 47.616519550427654, -122.13966755206604 47.616526111887509, -122.13968701903617 47.617280676597375, -122.142821316476 47.617300360798339, -122.142821316476 47.618186139853435, -122.14359028995352 47.618245191245848)))",1
1,100531,2,"GEOMETRYCOLLECTION (POINT (-122.142918 47.613462), POLYGON ((-122.14359028995352 47.618245191245848, -122.14360975757847 47.616519550427654, -122.13966755206604 47.616526111887509, -122.13968701903617 47.617280676597375, -122.142821316476 47.617300360798339, -122.142821316476 47.618186139853435, -122.14359028995352 47.618245191245848)))",0


You can copy a value for `WKT` above and plug it into a map to see that the `InGeoFence` column indeed matches if a bus is in the GeoFence. Note that `GEOMETRYCOLLECTION` allows you to plot points and polygons together.

## Part 3: Create Stored Procedures to get and add data

You've now seen how to use Azure SQL Database to determine if a bus is within a GeoFence. However, you now need to scale this so it can process real-time data as it flows in. Stored procedures will greatly simplify this in future exercises where you're leveraging other services, e.g. Azure Functions, Azure Logic Apps, Azure App Service, etc.

A stored procedure is a way to group SQL statements and execute them on the database with one command. For the catching the bus scenario, three stored procedures will be required and you will create them using your learnings from Parts 1 and 2. As you create the stored procedures, take some time to review the T-SQL and how it compares to what you learned in Parts 1 and 2.

1. **web.AddBusData**: this stored procedure takes in JSON data containing new bus route, vehicle, direction, time, and location information and adds it to the _busData_ table. If a bus enters/exits a GeoFence, it will also log this information in the _GeoFencesActive_ table.

In [ ]:
CREATE SCHEMA [web] AUTHORIZATION [dbo];
GO

In [4]:
/*
	Add received Bus geolocation data and check if buses are
	inside any defined GeoFence. JSON must be like:

	{
		"DirectionId": 1,
		"RouteId": 100001,
		"VehicleId": 2,
		"Position": {
			"Latitude": 47.61705102765316,
			"Longitude": -122.14291865504012 
		},
		"TimestampUTC": "20201031"
	}
}
*/
create or alter procedure [web].[AddBusData]
@payload nvarchar(max) 
as
begin	
	set nocount on
	set xact_abort on
	set tran isolation level serializable

	begin tran

	if (isjson(@payload) != 1) begin;
		throw 50000, 'Payload is not a valid JSON document', 16;
	end;

	declare @ids as table (id int);

	-- insert bus data
	insert into dbo.[BusData] 
		([DirectionId], [RouteId], [VehicleId], [Location], [TimestampUTC])
	output
		inserted.Id into @ids
	select
		[DirectionId], 
		[RouteId], 
		[VehicleId], 
		geography::Point([Latitude], [Longitude], 4326) as [Location], 
		[TimestampUTC]
	from
		openjson(@payload) with (
			[DirectionId] int,
			[RouteId] int,
			[VehicleId] int,
			[Latitude] decimal(10,6) '$.Position.Latitude',
			[Longitude] decimal(10,6) '$.Position.Longitude',
			[TimestampUTC] datetime2(7)
		);
		
	-- Get details of inserted data
	select * into #t from dbo.[BusData] bd where bd.id in (select i.id from @ids i);

	-- Find geofences in which the vehicle is in
	select 
		t.Id as BusDataId,
		t.[VehicleId],
		t.[DirectionId],
		t.[TimestampUTC],
		t.[RouteId],		
		g.Id as GeoFenceId
	into
		#g
	from 
		dbo.GeoFences g 
	right join
		#t t on g.GeoFence.STContains(t.[Location]) = 1;

	-- Calculate status
	select
		c.BusDataId,
		coalesce(a.[GeoFenceId], c.[GeoFenceId]) as GeoFenceId,
		coalesce(a.[DirectionId], c.[DirectionId]) as DirectionId,
		coalesce(a.[VehicleId], c.[VehicleId]) as VehicleId,
		c.[RouteId],
		c.[TimestampUTC],
		case 
			when a.GeoFenceId is null and c.GeoFenceId is not null then 'Enter'
			when a.GeoFenceId is not null and c.GeoFenceId is null then 'Exit'		
		end as [Status]
	into
		#s 
	from
		#g c
	full outer join
		dbo.GeoFencesActive a on c.DirectionId = a.DirectionId and c.VehicleId = a.VehicleId;
	
	-- Delete exited geofences
	delete 
		a
	from
		dbo.GeoFencesActive a
	inner join
		#s s on a.VehicleId = s.VehicleId and s.DirectionId = a.DirectionId and s.[Status] = 'Exit';

	-- Insert entered geofences
	insert into dbo.GeoFencesActive 
		([GeoFenceId], [DirectionId], [VehicleId])
	select
		[GeoFenceId], [DirectionId], [VehicleId]
	from
		#s s
	where 
		s.[Status] = 'Enter';

	-- Insert Log
	insert into dbo.GeoFenceLog 
		(GeoFenceId, BusDataId, [RouteId], [VehicleId], [TimestampUTC], [Status])
	select
		GeoFenceId, BusDataId, [RouteId], [VehicleId], [TimestampUTC], isnull([Status], 'In')
	from
		#s s
	where
		s.[GeoFenceId] is not null
	and
		s.[BusDataId] is not null

	-- Return Entered or Exited geofences
	select
	((
		select
			s.[BusDataId],  
			s.[VehicleId],
			s.[DirectionId],  
			s.[RouteId], 
			r.[ShortName] as RouteName,
			s.[GeoFenceId], 
			gf.[Name] as GeoFence,
			s.[Status] as GeoFenceStatus,
			s.[TimestampUTC]
		from
			#s s
		inner join
			dbo.[GeoFences] gf on s.[GeoFenceId] = gf.[Id]
		inner join
			dbo.[Routes] r on s.[RouteId] = r.[Id]
		where
			s.[Status] is not null and s.[GeoFenceId] is not null
		for 
			json path
	)) as ActivatedGeoFences;

	commit
end

Commands completed successfully.

Total execution time: 00:00:00.052

2. **web.GetMonitoredRoutes**: this stored procedure returns the route IDs for the bus routes that are being monitored.

In [5]:
/*
	Return the Routes (and thus the buses) to monitor
*/
create or alter procedure [web].[GetMonitoredRoutes]
as
begin
	select 
	((	
		select RouteId from dbo.[MonitoredRoutes] for json auto
	)) as MonitoredRoutes
end
GO

Commands completed successfully.

Total execution time: 00:00:00.045

3. **web.GetMonitoredBusData**: this stored procedure will return bus information for the 50 most-recent buses within 5 kilometers of the monitored GeoFence(s).

In [6]:
/*
	Return last geospatial data for bus closest to the GeoFence
*/
create or alter procedure [web].[GetMonitoredBusData]
@routeId int,
@geofenceId int
as
begin
	with cte as
	(
		-- Get the latest location of all the buses in the given route
		select top (1) with ties 
			*  
		from 
			dbo.[BusData] 
		where
			[RouteId] = @routeId
		order by 
			[ReceivedAtUTC] desc
	),
	cte2 as
	(
		-- Get the closest to the GeoFence
		select top (1)
			c.[VehicleId],
			gf.[GeoFence],
			c.[Location].STDistance(gf.[GeoFence]) as d
		from
			[cte] c
		cross join
			dbo.[GeoFences] gf
		where
			gf.[Id] = @geofenceId
		order by
			d 
	), cte3 as
	(
	-- Take the last 50 points 
	select top (50)
		[bd].[VehicleId],
		[bd].[DirectionId],
		[bd].[Location] as l,
		[bd].[Location].STDistance([GeoFence]) as d
	from
		dbo.[BusData] bd
	inner join
		cte2 on [cte2].[VehicleId] = [bd].[VehicleId]
	order by 
		id desc
	)
	-- Return only the points that are withing 5 Km
	select 
	((
		select
			geography::UnionAggregate(l).ToString() as [busData],
			(select [GeoFence].ToString() from dbo.[GeoFences] where Id = @geofenceId) as [geoFence]
		from
			cte3
		where
			d < 5000
		for json auto, include_null_values, without_array_wrapper
	)) as locationData
end
GO

Commands completed successfully.

Total execution time: 00:00:00.050

Confirm you've created the stored procedures with the following.

In [9]:
SELECT * FROM INFORMATION_SCHEMA.ROUTINES WHERE ROUTINE_SCHEMA = 'web'

(3 rows affected)

Total execution time: 00:00:00.083

SPECIFIC_CATALOG,SPECIFIC_SCHEMA,SPECIFIC_NAME,ROUTINE_CATALOG,ROUTINE_SCHEMA,ROUTINE_NAME,ROUTINE_TYPE,MODULE_CATALOG,MODULE_SCHEMA,MODULE_NAME,UDT_CATALOG,UDT_SCHEMA,UDT_NAME,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,CHARACTER_OCTET_LENGTH,COLLATION_CATALOG,COLLATION_SCHEMA,COLLATION_NAME,CHARACTER_SET_CATALOG,CHARACTER_SET_SCHEMA,CHARACTER_SET_NAME,NUMERIC_PRECISION,NUMERIC_PRECISION_RADIX,NUMERIC_SCALE,DATETIME_PRECISION,INTERVAL_TYPE,INTERVAL_PRECISION,TYPE_UDT_CATALOG,TYPE_UDT_SCHEMA,TYPE_UDT_NAME,SCOPE_CATALOG,SCOPE_SCHEMA,SCOPE_NAME,MAXIMUM_CARDINALITY,DTD_IDENTIFIER,ROUTINE_BODY,ROUTINE_DEFINITION,EXTERNAL_NAME,EXTERNAL_LANGUAGE,PARAMETER_STYLE,IS_DETERMINISTIC,SQL_DATA_ACCESS,IS_NULL_CALL,SQL_PATH,SCHEMA_LEVEL_ROUTINE,MAX_DYNAMIC_RESULT_SETS,IS_USER_DEFINED_CAST,IS_IMPLICITLY_INVOCABLE,CREATED,LAST_ALTERED
bus-db,web,AddBusData,bus-db,web,AddBusData,PROCEDURE,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,SQL,"/* Add received Bus geolocation data and check if buses are inside any defined GeoFence. JSON must be like: { ""DirectionId"": 1, ""RouteId"": 100001, ""VehicleId"": 2, ""Position"": { ""Latitude"": 47.61705102765316, ""Longitude"": -122.14291865504012 }, ""TimestampUTC"": ""20201031"" } } */ create procedure [web].[AddBusData] @payload nvarchar(max) as begin set nocount on set xact_abort on set tran isolation level serializable begin tran if (isjson(@payload) != 1) begin; throw 50000, 'Payload is not a valid JSON document', 16; end; declare @ids as table (id int); -- insert bus data insert into dbo.[BusData] ([DirectionId], [RouteId], [VehicleId], [Location], [TimestampUTC]) output inserted.Id into @ids select [DirectionId], [RouteId], [VehicleId], geography::Point([Latitude], [Longitude], 4326) as [Location], [TimestampUTC] from openjson(@payload) with ( [DirectionId] int, [RouteId] int, [VehicleId] int, [Latitude] decimal(10,6) '$.Position.Latitude', [Longitude] decimal(10,6) '$.Position.Longitude', [TimestampUTC] datetime2(7) ); -- Get details of inserted data select * into #t from dbo.[BusData] bd where bd.id in (select i.id from @ids i); -- Find geofences in which the vehicle is in select t.Id as BusDataId, t.[VehicleId], t.[DirectionId], t.[TimestampUTC], t.[RouteId], g.Id as GeoFenceId into #g from dbo.GeoFences g right join #t t on g.GeoFence.STContains(t.[Location]) = 1; -- Calculate status select c.BusDataId, coalesce(a.[GeoFenceId], c.[GeoFenceId]) as GeoFenceId, coalesce(a.[DirectionId], c.[DirectionId]) as DirectionId, coalesce(a.[VehicleId], c.[VehicleId]) as VehicleId, c.[RouteId], c.[TimestampUTC], case when a.GeoFenceId is null and c.GeoFenceId is not null then 'Enter' when a.GeoFenceId is not null and c.GeoFenceId is null then 'Exit' end as [Status] into #s from #g c full outer join dbo.GeoFencesActive a on c.DirectionId = a.DirectionId and c.VehicleId = a.VehicleId; -- Delete exited geofences delete a from dbo.GeoFencesActive a inner join #s s on a.VehicleId = s.VehicleId and s.DirectionId = a.DirectionId and s.[Status] = 'Exit'; -- Insert entered geofences insert into dbo.GeoFencesActive ([GeoFenceId], [DirectionId], [VehicleId]) select [GeoFenceId], [DirectionId], [VehicleId] from #s s where s.[Status] = 'Enter'; -- Insert Log insert into dbo.GeoFenceLog (GeoFenceId, BusDataId, [RouteId], [VehicleId], [TimestampUTC], [Status]) select GeoFenceId, BusDataId, [RouteId], [VehicleId], [TimestampUTC], isnull([Status], 'In') from #s s where s.[GeoFenceId] is not null and s.[BusDataId] is not null -- Return Entered or Exited geofences select (( select s.[BusDataId], s.[VehicleId], s.[DirectionId], s.[RouteId], r.[ShortName] as RouteName, s.[GeoFenceId], gf.[Name] as GeoFence, s.[Status] as GeoFenceStatus, s.[TimestampUTC] from #s s inner join dbo.[GeoFences] gf on s.[GeoFenceId] = gf.[Id] inner join dbo.[Routes] r on s.[RouteId] = r.[Id] where s.[Status] is not null and s.[GeoFenceId] is not null for json pa